In [ ]:
import os
import rospkg
import numpy as np
import tf

# Create a RosPack object
rospack = rospkg.RosPack()

# Get the path to the package this script is in
package_path = rospack.get_path('hri_predict_ros')

# Define the path to the plots directory
plot_dir = os.path.join(package_path, 'plots')
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

# Specify which topics to read from the rosbag file
topic_names = ['/offline/zed/zed_node/body_trk/skeletons']

n_kpts = 18
TF_world_camera = [0.100575, -0.9304, 2.31042, 0.180663, 0.516604, 0.119341, 0.828395]

translation_world_camera = np.array(TF_world_camera[0:3])
quaternion_world_camera = np.array(TF_world_camera[3:7])

# Convert the quaternion to a rotation matrix
rotation_matrix_world_camera = tf.transformations.quaternion_matrix(quaternion_world_camera)

# Create a translation matrix
translation_matrix_world_camera = tf.transformations.translation_matrix(translation_world_camera)

# Combine the rotation and translation to get the transformation matrix from the world frame to the camera frame
cam_to_world_matrix = tf.transformations.concatenate_matrices(
    translation_matrix_world_camera,
    rotation_matrix_world_camera
)

human_meas_names = ['human_kp{}_{}'.format(i, suffix)
                    for i in range(n_kpts)
                    for suffix in ['x', 'y', 'z']]

# Define the frequency of the measurements for resampling
f = 20 # Hz
meas_dt = 1/f
freq_str = f'{meas_dt}S' # seconds

Import measurements from bag file

In [ ]:
import rosbag

# Define the path to the bag directory
bag_dir = os.path.join(package_path, 'logs', 'bag')

bag_files = os.listdir(bag_dir)
bag_files = [os.path.join(bag_dir, bag_file) for bag_file in bag_files]

bag_data = {}
for bag_file in bag_files:
    with rosbag.Bag(bag_file, 'r') as bag:
        rows_list = []
        for topic, msg, t in bag.read_messages(topics=topic_names):
            row_dict = {}

            timestamp = t.to_sec()

            human_meas = np.full((1, n_kpts*3), np.nan)
            if topic == '/offline/zed/zed_node/body_trk/skeletons':
                skeleton_kpts = np.full((n_kpts, 3), np.nan)
                if msg.objects:                
                    for obj in msg.objects:
                        # Extract skeleton keypoints from message ([x, y, z] for each kpt)
                        kpts = np.array([[kp.kp] for kp in obj.skeleton_3d.keypoints])
                        kpts = kpts[:n_kpts] # select only the first n_kpts

                        skeleton_kpts = np.reshape(kpts, (n_kpts, 3)) # reshape to (n_kpts, 3)

                        # Convert keypoints to world frame
                        for i in range(n_kpts):
                            # Create a homogeneous coordinate for the keypoint position
                            kpt = np.array([skeleton_kpts[i][0],
                                            skeleton_kpts[i][1],
                                            skeleton_kpts[i][2],
                                            1])

                            # Transform the keypoint to the world frame using the transformation matrix
                            kpt_world = np.dot(cam_to_world_matrix, kpt)

                            skeleton_kpts[i][0] = kpt_world[0]
                            skeleton_kpts[i][1] = kpt_world[1]
                            skeleton_kpts[i][2] = kpt_world[2]
                    
                else:
                    skeleton_kpts = np.full(skeleton_kpts.shape, np.nan)

                # Update current human measurement vector
                human_meas = skeleton_kpts.flatten()

            row_dict.update({'timestamp': timestamp})
            row_dict.update({'human_meas': human_meas.flatten()})

            rows_list.append(row_dict)

    subject_id = bag_file.split('.')[0].split('simple_')[-1]
    bag_data[subject_id] = rows_list

Store measurement data in a dictionary of Pandas dataframes

In [ ]:
import pandas as pd

measurement_data = {}
for subject, bag in bag_data.items():
    data = pd.DataFrame(bag, columns=['timestamp', 'human_meas'])

    # split columns into separate columns
    for c in data.columns.values:
        data = pd.concat([data, data.pop(c).apply(pd.Series).add_prefix(c+"_")], axis=1)

    # change column names
    data.columns = ['timestamp'] + human_meas_names

    # Make time index relative to the start of the recording
    # data['timestamp'] = data['timestamp'] - data['timestamp'][0]

    # Convert the 'timestamp' column to a TimeDeltaIndex
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')

    # Increase the timestamp by 2 hours to match the system time
    data['timestamp'] = data['timestamp'] + pd.Timedelta(hours=2)

    # Resample the DataFrame to a known frequency
    resampled_data = data.resample(freq_str, on='timestamp').mean()

    measurement_data[subject] = data

for key, value in measurement_data.items():
    print(f'{key}: \t\t{value}')

In [ ]:
import plotly.express as px

example_data = measurement_data['sub_7']

fig = px.scatter(example_data, x=example_data.index, y=['human_kp4_y'])

fig.update_traces(marker=dict(size=3))
fig.update_layout(title='kp4_y', xaxis_title='Timestamp', yaxis_title='Value')
fig.update_xaxes(tickformat='%H:%M:%S')

fig.show()

Import task data from GUI logs

In [ ]:
gui_dir = os.path.join(package_path, 'logs', 'gui_data')

# Get all the file names in the directory
gui_files = os.listdir(gui_dir)

gui_data = {}
for gui_file in gui_files:
    # Check if the file is a text file
    if gui_file.endswith('.txt'): # they are txt files, but structured as csv
        # Construct the file path
        file_path = os.path.join(gui_dir, gui_file)
        
        # Read the file as a dataframe
        df = pd.read_csv(file_path)
        
        # Add the dataframe to the dictionary using a portion of the file name as the key
        key = gui_file.split('.')[0].split('_')[-2:]
        key = '_'.join(key)
        gui_data[key] = df

Define ranges from gui data to split measurements in tasks

In [ ]:
VELOCITIES = ['SLOW', 'MEDIUM', 'FAST']
TASK_NAMES = ['PICK-&-PLACE', 'WALKING', 'PASSING-BY']

trigger_data = {}
for subject, gui in gui_data.items():
    for velocity in VELOCITIES:
        for task in TASK_NAMES:
            trigger_data[(subject, velocity, task)] = gui.loc[(gui['Velocity'] == velocity) & (gui['Task_name'] == task)]

            # only keep the first and last timestamp of each trigger_data
            first_last_timestamps = trigger_data[(subject, velocity, task)].iloc[[0, -1]]['Timestamp'].values
            trigger_data[(subject, velocity, task)] = first_last_timestamps

for key, value in trigger_data.items():
    print(f'{key}: \t\t{value}')

Define helper functions for Kalman Filtering

In [89]:
def get_near_psd(P, max_iter=10):

    eps = 1e-3  # Small positive jitter for regularization
    increment_factor = 10  # Factor to increase eps if needed
        
    def is_symmetric(A):
        return np.allclose(A, A.T)
                    
    def is_positive_definite(A):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
        
    for _ in range(max_iter):
        if is_symmetric(P) and is_positive_definite(P):
            return P  # The matrix is suitable for Cholesky
    
        # Make P symmetric
        P = (P + P.T) / 2
    
        # Set negative eigenvalues to zero
        eigval, eigvec = np.linalg.eig(P)
        eigval[eigval < 0] = 0
        # add a jitter for strictly positive
        eigval += eps
    
        # Reconstruct the matrix
        P = eigvec.dot(np.diag(eigval)).dot(eigvec.T)

        # Force P to be real
        P = np.real(P)

        # Force off-diagonal elements to be zero => do it since the keypoints are all independent
        P = np.diag(np.diag(P))
    
        # Check if P is now positive definite
        if is_positive_definite(P):
            return P
    
        # Increase regularization factor for the next iteration
        eps *= increment_factor
    
    raise ValueError("Unable to convert the matrix to positive definite within max iterations.")

Define parameters for the filters

In [93]:
from filterpy.kalman import UnscentedKalmanFilter, MerweScaledSigmaPoints
from filterpy.kalman import IMMEstimator
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
import copy, time
from tqdm import tqdm

# Parameters
dt = 0.1
k = 5
predict_k_steps = False
n_kpts = 18
n_var_per_dof = 3       # position, velocity, acceleration
n_dim_per_kpt = 3       # x, y, z
var_r = 0.0025
var_q = 0.000025
init_P = 1.0
max_time_no_meas = pd.Timedelta(seconds=1.0)

# Transition matrix for IMM
M = np.array([[0.55, 0.15, 0.30],
              [0.15, 0.75, 0.10],
              [0.60, 0.30, 0.10]])
mu = np.array([0.45, 0.30, 0.25])

# Subject IDS
subject_ids = gui_data.keys()
print("Subjects: ", subject_ids)

# Split subjects into train and test
train_subjects = ['sub_7', 'sub_8']
test_subjects = ['sub_9', 'sub_12', 'sub_13']

# Dimensions
dim_x = n_var_per_dof * n_dim_per_kpt * n_kpts # 3D (position, velocity, acceleration) for each keypoint
dim_z = n_dim_per_kpt * n_kpts # 3D position for each keypoint

# Position indices
p_idx = np.arange(0, dim_x, n_var_per_dof)

# Column names
state_names = ['kp{}_{}'.format(i, suffix)
               for i in range(n_kpts)
               for suffix in ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd']]
measurement_names = ['kp{}_{}'.format(i, suffix)
                     for i in range(n_kpts)
                     for suffix in ['x', 'y', 'z']]
filtered_column_names = ['{}_kp{}_{}'.format(filt_type, i, suffix)
                         for filt_type in ['ca', 'cv', 'imm']
                         for i in range(n_kpts)
                         for suffix in ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd']]
filtered_pred_column_names = ['{}_kp{}_{}'.format(filt_type, i, suffix)
                              for filt_type in ['ca', 'imm']
                              for i in range(n_kpts)
                              for suffix in ['x', 'xd', 'xdd', 'y', 'yd', 'ydd', 'z', 'zd', 'zdd']]
col_names_imm = ['imm_pos', 'imm_vel', 'imm_acc']
col_names_prob_imm = ['prob_ca', 'prob_ca_no', 'prob_cv']

# measurement function: only the position is measured
def hx(x):
    return x[p_idx]

sigmas = MerweScaledSigmaPoints(n=dim_x, alpha=.1, beta=2., kappa=1.)

# CONSTANT ACCELERATION UKF
F_block_ca = np.array([[1, dt, 0.5*dt**2],
                       [0, 1, dt],
                       [0, 0, 1]])
F_ca = block_diag(*[F_block_ca for _ in range(n_dim_per_kpt * n_kpts)])

# state transition function: const acceleration
def fx_ca(x, dt):
    return np.dot(F_ca, x)

# CONSTANT VELOCITY UKF
F_block_cv = np.array([[1, dt, 0],
                       [0, 1, 0],
                       [0, 0, 0]])
F_cv = block_diag(*[F_block_cv for _ in range(n_dim_per_kpt * n_kpts)])

# state transition function: const velocity
def fx_cv(x, dt):
    return np.dot(F_cv, x)

Subjects:  dict_keys(['sub_7', 'sub_9', 'sub_13', 'sub_8', 'sub_12'])


Execute predict-update-k_step_predict loop:
- for each subject
- for each velocity
- for each task

and aggregate results

In [95]:
def run_filtering_loop(subject_ids, velocities, task_names):
    # CONSTANT ACCELERATION UKF
    ca_ukf = UnscentedKalmanFilter(dim_x=dim_x, dim_z=dim_z, dt=dt, hx=hx, fx=fx_ca, points=sigmas)
    ca_ukf.x = np.nan * np.ones(dim_x)
    ca_ukf.P = np.eye(dim_x) * init_P
    ca_ukf.R = var_r
    ca_ukf.Q = Q_discrete_white_noise(dim=n_var_per_dof, dt=dt, var=var_q, block_size=n_dim_per_kpt * n_kpts)
    uxs_ca = []

    # CONSTANT ACCELERATION UKF WITH NO PROCESS ERROR
    ca_no_ukf = copy.deepcopy(ca_ukf)
    ca_no_ukf.Q = np.zeros((dim_x, dim_x))

    # CONSTANT VELOCITY UKF
    cv_ukf = UnscentedKalmanFilter(dim_x=dim_x, dim_z=dim_z, dt=dt, hx=hx, fx=fx_cv, points=sigmas)
    cv_ukf.x = np.nan * np.ones(dim_x)
    cv_ukf.P = np.eye(dim_x) * init_P
    cv_ukf.R = var_r
    cv_ukf.Q = Q_discrete_white_noise(dim=n_var_per_dof, dt=dt, var=var_q, block_size=n_dim_per_kpt * n_kpts)
    uxs_cv = []

    # IMM ESTIMATOR
    filters = [copy.deepcopy(ca_ukf), ca_no_ukf, copy.deepcopy(cv_ukf)]

    bank = IMMEstimator(filters, mu, M)
    uxs_bank, probs_bank = [], []

    # K-STEP AHEAD PREDICTION FILTERS
    ca_ukf_pred = copy.deepcopy(ca_ukf)
    uxs_ca_pred = []
    bank_pred = copy.deepcopy(bank)
    uxs_bank_pred = []
    probs_bank_pred = []

    # Create dictionary to store results
    measurement_split = {}
    filtering_results = {}

    for subject_id in subject_ids:
        for velocity in velocities:
            for task in task_names:
                print(f'Filtering subject {subject_id} - {velocity} - {task}')

                # Get the trigger timestamps for the current task
                trigger_timestamps = trigger_data[(subject_id, velocity, task)]

                # Get only the measurements whose timestamps are within the trigger timestamps
                start_trigger = pd.to_datetime(trigger_timestamps[0])
                end_trigger = pd.to_datetime(trigger_timestamps[1])

                print("Selecting measurements from: ", start_trigger, "to", end_trigger)

                zs = measurement_data[subject_id].loc[(measurement_data[subject_id]['timestamp'] >= start_trigger) &
                                                    (measurement_data[subject_id]['timestamp'] <= end_trigger)]
                zs.set_index('timestamp', inplace=True)

                # Resample the measurements to a known frequency and subtract initial time
                zs = zs.resample(freq_str).mean()
                zs.index = zs.index - zs.index[0]
                
                measurement_split[(subject_id, velocity, task)] = zs

                # Define times
                t = zs.index[0]
                t_end = zs.index[-1]
                t_incr = pd.Timedelta(seconds=dt)

                print("Start time:", t, "End time:", t_end)

                # Initialization flag
                time_no_meas = pd.Timedelta(seconds=0)
                ufk_initialized = False
                filt_timestamps = []
                elapsed_time = 0.0

                # Main loop
                total_iterations = int((t_end - t) / t_incr) + 1
                pbar = tqdm(total=total_iterations)
                while t <= t_end:
                    tic = time.time()
                    filt_timestamps.append(t)

                    if t in zs.index:
                        z = np.array(zs.loc[t])

                    measure_received = t in zs.index and not np.isnan(z).any()

                    if measure_received and not ufk_initialized:
                        # print('timestamp:', t, 'measure:', z, 'initializing filters')
                        # initial state: [pos, vel, acc] = [current measured position, 0.0, 0.0]
                        ca_ukf.x = np.zeros(dim_x)
                        ca_ukf.x[p_idx] = z
                        cv_ukf.x = np.zeros(dim_x)
                        cv_ukf.x[p_idx] = z
                        for f in bank.filters:
                            f.x = np.zeros(dim_x)
                            f.x[p_idx] = z
                        ufk_initialized = True

                    else:
                        if not measure_received and ufk_initialized:
                            time_no_meas += t_incr
                            # print('timestamp:', t, 'no measure received for', time_no_meas, 'seconds')

                        if time_no_meas >= max_time_no_meas:
                            ufk_initialized = False
                        
                            # Reset filter states
                            ca_ukf.x = np.nan * np.ones(dim_x)
                            cv_ukf.x = np.nan * np.ones(dim_x)
                            bank.x = np.nan * np.ones(dim_x)
                            if predict_k_steps:
                                ca_ukf_pred.x = np.nan * np.ones(dim_x)
                                bank_pred.x = np.nan * np.ones(dim_x)

                            # Reset filter covariances
                            ca_ukf.P = np.eye(dim_x) * init_P
                            cv_ukf.P = np.eye(dim_x) * init_P
                            bank.P = np.eye(dim_x) * init_P
                            if predict_k_steps:
                                ca_ukf_pred.P = np.eye(dim_x) * init_P
                                bank_pred.P = np.eye(dim_x) * init_P
                        
                    if ufk_initialized:
                        # make sure covariance matrices are positive semidefinite
                        ca_ukf.P = get_near_psd(ca_ukf.P)
                        cv_ukf.P = get_near_psd(cv_ukf.P)
                        for f in bank.filters:
                            f.P = get_near_psd(f.P)

                        ca_ukf.predict()
                        cv_ukf.predict()
                        bank.predict()
                        if measure_received:
                            time_no_meas = pd.Timedelta(seconds=0)
                            ca_ukf.update(z)
                            cv_ukf.update(z)
                            bank.update(z)

                        if predict_k_steps:
                            # Predict k steps ahead starting from the current state and covariance
                            ca_ukf_pred.x = ca_ukf.x.copy()
                            ca_ukf_pred.P = ca_ukf.P.copy()
                            bank_pred.x = bank.x.copy()
                            for f_pred, f in zip(bank_pred.filters, bank.filters):
                                f_pred.x = f.x.copy()
                                f_pred.P = f.P.copy()
                                
                            for i in range(k):
                                # make sure covariance matrices are positive semidefinite
                                ca_ukf_pred.P = get_near_psd(ca_ukf_pred.P)
                                for f in bank_pred.filters:
                                    f.P = get_near_psd(f.P)

                                ca_ukf_pred.predict()
                                bank_pred.predict()
                        
                    uxs_ca.append(ca_ukf.x.copy())
                    uxs_cv.append(cv_ukf.x.copy())
                    uxs_bank.append(bank.x.copy())
                    probs_bank.append(bank.mu.copy())

                    if predict_k_steps:
                        uxs_ca_pred.append(ca_ukf_pred.x.copy())
                        uxs_bank_pred.append(bank_pred.x.copy())
                        probs_bank_pred.append(bank_pred.mu.copy())

                    t += t_incr
                    toc = time.time()
                    elapsed_time += (toc - tic)
                    pbar.update()

                pbar.close()

                # Convert the lists to numpy arrays
                uxs_ca = np.array(uxs_ca)
                uxs_cv = np.array(uxs_cv)
                uxs_bank = np.array(uxs_bank)
                uxs = np.concatenate((uxs_ca, uxs_cv, uxs_bank), axis=1)
                probs_bank = np.array(probs_bank)

                if predict_k_steps:
                    uxs_ca_pred = np.array(uxs_ca_pred)
                    uxs_bank_pred = np.array(uxs_bank_pred)
                    uxs_pred = np.concatenate((uxs_ca_pred, uxs_bank_pred), axis=1)
                    probs_bank_pred = np.array(probs_bank_pred)

                # Create DataFrames with the filtered data
                filtered_data = pd.DataFrame(uxs, index=filt_timestamps, columns=filtered_column_names)
                imm_probs = pd.DataFrame(probs_bank, index=filt_timestamps, columns=col_names_prob_imm)

                if predict_k_steps:
                    # Create DataFrames with the k-step ahead prediction data
                    kstep_pred_data = pd.DataFrame(uxs_pred, index=filt_timestamps, columns=filtered_pred_column_names)
                    kstep_pred_imm_probs = pd.DataFrame(probs_bank_pred, index=filt_timestamps, columns=col_names_prob_imm)

                    # Shift the k-step ahead prediction data by k steps
                    kstep_pred_data = kstep_pred_data.shift(+k)
                    kstep_pred_imm_probs = kstep_pred_imm_probs.shift(+k)

                if predict_k_steps: 
                    # Store filtering results
                    filtering_results[(subject_id, velocity, task)] = {
                        'filtered_data': filtered_data,
                        'imm_probs': imm_probs,
                        'kstep_pred_data': kstep_pred_data,
                        'kstep_pred_imm_probs': kstep_pred_imm_probs
                    }
                else:
                    # Store filtering results
                    filtering_results[(subject_id, velocity, task)] = {
                        'filtered_data': filtered_data,
                        'imm_probs': imm_probs
                    }

                print("Mean loop frequency: {:.2f} Hz".format(1.0 / (elapsed_time / len(filt_timestamps))))
                print(f"Processed {subject_id} - {velocity} - {task}")

    return measurement_split, filtering_results

In [ ]:
measurement_split, filtering_results = run_filtering_loop(['sub_7'], ['SLOW'], ['PICK-&-PLACE'])

Plot some results

In [84]:
subject = 'sub_7'
velocity = 'SLOW'
task = 'PICK-&-PLACE'
state = 'kp4_x'
# state = 'kp0_y'
description = 'Right-hand wrist keypoint x-coordinate'
# description = "Neck keypoint y-coordinate"

meas = measurement_split[(subject, velocity, task)]
meas_seconds = (meas.index - meas.index[0]).total_seconds()

filt = filtering_results[(subject, velocity, task)]['filtered_data']
filt_seconds = (filt.index - filt.index[0]).total_seconds()

if predict_k_steps:
    kpred = filtering_results[(subject, velocity, task)]['kstep_pred_data']
    kpred_seconds = (kpred.index - kpred.index[0]).total_seconds()

fig = px.line()
fig.add_scatter(x=meas_seconds,
                y=meas['_'.join(('human',state))],
                mode='lines+markers',
                name='Measurements'
)
fig.add_scatter(x=filt_seconds,
                y=filt['_'.join(('ca',state))],
                mode='lines+markers',
                name='UKF CA'
)
fig.add_scatter(x=filt_seconds,
                y=filt['_'.join(('cv',state))],
                mode='lines+markers',
                name='UKF CV'
)
fig.add_scatter(x=filt_seconds,
                y=filt['_'.join(('imm',state))],
                mode='lines+markers',
                name='UKF IMM'
)
if predict_k_steps:
    fig.add_scatter(x=kpred_seconds,
                    y=kpred['_'.join(('ca',state))],
                    mode='lines+markers',
                    name='CA K-step ahead prediction'
    )
    fig.add_scatter(x=kpred_seconds,
                    y=kpred['_'.join(('imm',state))],
                    mode='lines+markers',
                    name='IMM K-step ahead prediction'
    )

fig.update_traces(marker=dict(size=2), line=dict(width=1))
fig.update_layout(title=description+" position"+f" [{subject}, {velocity}, {task}] "+f" (dt={dt}, k={k})",
                  xaxis_title='Time (s)',
                  yaxis_title='Position (m)')

fig.show()

# Save the plot to the plots folder in html format
plot_name = '_'.join([subject, velocity, task, state, "dt", str(dt), "k", str(k), "pos"]) + '.html'
fig.write_html(os.path.join(plot_dir, plot_name))

In [86]:
state = 'kp0_yd'

fig = px.line()
fig.add_scatter(x=filt_seconds,
                y=filt['_'.join(('ca',state))],
                mode='lines+markers',
                name='UKF CA'
)
fig.add_scatter(x=filt_seconds,
                y=filt['_'.join(('cv',state))],
                mode='lines+markers',
                name='UKF CV'
)
fig.add_scatter(x=filt_seconds,
                y=filt['_'.join(('imm',state))],
                mode='lines+markers',
                name='UKF IMM'
)
if predict_k_steps:
    fig.add_scatter(x=kpred_seconds,
                    y=kpred['_'.join(('ca',state))],
                    mode='lines+markers',
                    name='CA K-step ahead prediction'
    )
    fig.add_scatter(x=kpred_seconds,
                    y=kpred['_'.join(('imm',state))],
                    mode='lines+markers',
                    name='IMM K-step ahead prediction'
    )

fig.update_traces(marker=dict(size=2), line=dict(width=1))
fig.update_layout(title=description+" velocity"+f" [{subject}, {velocity}, {task}] "+f" (dt={dt}, k={k})",
                  xaxis_title='Time (s)',
                  yaxis_title='Velocity (m/s)')

fig.show()

# Save the plot to the plots folder in html format
plot_name = '_'.join([subject, velocity, task, state, "dt", str(dt), "k", str(k), "vel"]) + '.html'
fig.write_html(os.path.join(plot_dir, plot_name))